# Dynamic Modeling in Behavorial Ecology: Julia Edition

In which we work through several examples from Mangel & Clark in Julia.

In [1]:
# patches are characterized the by following four parameters
type Patch
    β::Float64 # MC's probability of predation
    λ::Float64 # MC's probability of finding food
    α::Int64 # MC's cost to visit
    Y::Int64 # MC's value of food
end

In [2]:
# an Ecosystem is essentially an array of patches
typealias Ecosystem Array{Patch,1};

In [4]:
# the patches from page 54
myplace = Ecosystem([
    Patch(0., 0., 1, 0),
    Patch(0.004, 0.4, 1, 3), 
    Patch(0.02, 0.6, 1, 5)
    ]);

x_c = 3::Int64; # critical energy threshold
C = 10::Int64; # max capacity
T = 20::Int64; # time horizon

20

In [5]:
function xprime(x::Int64, α::Int64, y::Int64, x_critical::Int64, C::Int64)
    return clamp(x - α + y, x_critical, C)
end;

function xprimeprime(x::Int64, α::Int64, x_critical::Int64, C::Int64)
    return clamp(x - α, x_critical, C)
end;


In [6]:
function findOptimalSelections(
    aplace::Ecosystem,
    C::Int64,
    T::Int64,
    x_c::Int64)
    Fitness = zeros(Float64, C+1, T)
    Selection = zeros(Int64, C+1, T)
    for t = T:-1:1
        for x = 0:C
            if t == T
                Fitness[x+1, t] = x > x_c ? 1.0 : 0.0
            elseif x <= x_c 
                Fitness[x+1, t] = 0.0
            else 
                patchoptions = [(1-P.β)*
                    (P.λ*Fitness[xprime(x, P.α, P.Y, x_c, C)+1,t+1] +
                    (1.0-P.λ)*Fitness[xprimeprime(x, P.α, x_c, C)+1,t+1])
                    for P in aplace]
                (Fitness[x+1, t], Selection[x+1, t]) = findmax(patchoptions)
            end
        end
    end
    return Selection, Fitness
end;

In [7]:
(Selection, Fitness) = findOptimalSelections(myplace, C, T, x_c);

In [8]:
## compare to Mangel and Clark page 55
Fitness[5:11,19:-1:1]

7x19 Array{Float64,2}:
 0.588  0.588     0.588     0.588     …  0.5227    0.518155  0.513651
 1.0    0.818496  0.818496  0.818496     0.736511  0.730198  0.723863
 1.0    1.0       0.90885   0.90885      0.825702  0.818574  0.811538
 1.0    1.0       1.0       0.944269     0.861025  0.853536  0.846181
 1.0    1.0       1.0       1.0          0.881215  0.873557  0.865991
 1.0    1.0       1.0       1.0       …  0.893367  0.885622  0.877955
 1.0    1.0       1.0       1.0          0.901125  0.893367  0.885622